In [1]:
import pandas as pd
import sqlite3
import numpy as np

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("SELECT * FROM fbref_match_summary_v2", conn)

df

,team,is_home,match_date,division,season,match_url,summary_minutes,summary_goals,summary_assists,summary_pens_made,...,opp_defense_challenge_tackles,opp_defense_challenges,opp_defense_challenge_tackles_pct,opp_defense_challenges_lost,opp_defense_blocked_shots,opp_defense_blocked_passes,opp_defense_tackles_interceptions,opp_defense_clearances,opp_defense_errors,opp_keeper_psxg
0,Tottenham,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,990.0,1.0,0.0,1.0,...,7.0,11.0,63.6,4.0,1.0,11.0,25.0,8.0,0.0,4.1
1,Brighton,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,990.0,4.0,1.0,1.0,...,5.0,15.0,33.3,10.0,7.0,4.0,24.0,41.0,1.0,1.5
2,Bournemouth,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,990.0,2.0,2.0,0.0,...,9.0,19.0,47.4,10.0,6.0,8.0,24.0,31.0,1.0,0.0
3,Leicester City,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,990.0,0.0,0.0,0.0,...,5.0,10.0,50.0,5.0,0.0,4.0,22.0,11.0,0.0,2.1
4,Newcastle Utd,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/36844e73/Newcastl...,990.0,0.0,0.0,0.0,...,7.0,12.0,58.3,5.0,4.0,2.0,27.0,23.0,0.0,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41879,Cagliari,0,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/e8e75a75/Como-Cag...,990.0,1.0,1.0,0.0,...,11.0,17.0,64.7,6.0,2.0,3.0,27.0,22.0,0.0,1.4
41880,Cittadella,1,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/99979e25/Cittadel...,990.0,4.0,1.0,2.0,...,1.0,7.0,14.3,6.0,4.0,3.0,27.0,19.0,0.0,1.4
41881,Pisa,0,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/99979e25/Cittadel...,990.0,3.0,2.0,0.0,...,5.0,13.0,38.5,8.0,2.0,3.0,25.0,13.0,0.0,2.2
41882,Parma,1,2022-08-12,Serie B,2022-2023,https://fbref.com/en/matches/be681acc/Parma-Ba...,990.0,2.0,1.0,0.0,...,5.0,16.0,31.3,11.0,1.0,3.0,24.0,22.0,0.0,2.1


In [2]:
promoted_teams = pd.read_csv("promoted_teams.csv")

promoted_teams

,team,division,season
0,Newcastle Utd,Premier League,2017-2018
1,Brighton,Premier League,2017-2018
2,Huddersfield,Premier League,2017-2018
3,Wolves,Premier League,2018-2019
4,Cardiff City,Premier League,2018-2019
...,...,...,...
141,Valladolid,La Liga,2022-2023
142,Girona,La Liga,2022-2023
143,Leganés,La Liga,2024-2025
144,Valladolid,La Liga,2024-2025


In [3]:
df['gw'] = df.groupby(['team', 'season', 'division'])['match_date'].rank(method='dense').astype(int)
df['is_early_season?'] = np.where(df['gw'] < 10, 1, 0)
df['is_promoted?'] = df.merge(promoted_teams, on=['team', 'division', 'season'], how='left', indicator=True)['_merge'].eq('both').astype(int)
df['opp_is_promoted?'] = df.merge(promoted_teams, left_on=['opp_team', 'division', 'season'], right_on=['team', 'division', 'season'], how='left', indicator=True)['_merge'].eq('both').astype(int)

df

,team,is_home,match_date,division,season,match_url,summary_minutes,summary_goals,summary_assists,summary_pens_made,...,opp_defense_blocked_shots,opp_defense_blocked_passes,opp_defense_tackles_interceptions,opp_defense_clearances,opp_defense_errors,opp_keeper_psxg,gw,is_early_season?,is_promoted?,opp_is_promoted?
0,Tottenham,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,990.0,1.0,0.0,1.0,...,1.0,11.0,25.0,8.0,0.0,4.1,38,0,0,0
1,Brighton,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/e4bb1c35/Tottenha...,990.0,4.0,1.0,1.0,...,7.0,4.0,24.0,41.0,1.0,1.5,38,0,0,0
2,Bournemouth,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,990.0,2.0,2.0,0.0,...,6.0,8.0,24.0,31.0,1.0,0.0,38,0,0,1
3,Leicester City,0,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/1ff370e8/Bournemo...,990.0,0.0,0.0,0.0,...,0.0,4.0,22.0,11.0,0.0,2.1,38,0,1,0
4,Newcastle Utd,1,2025-05-25,Premier League,2024-2025,https://fbref.com/en/matches/36844e73/Newcastl...,990.0,0.0,0.0,0.0,...,4.0,2.0,27.0,23.0,0.0,1.1,38,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41879,Cagliari,0,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/e8e75a75/Como-Cag...,990.0,1.0,1.0,0.0,...,2.0,3.0,27.0,22.0,0.0,1.4,1,1,0,0
41880,Cittadella,1,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/99979e25/Cittadel...,990.0,4.0,1.0,2.0,...,4.0,3.0,27.0,19.0,0.0,1.4,1,1,0,0
41881,Pisa,0,2022-08-13,Serie B,2022-2023,https://fbref.com/en/matches/99979e25/Cittadel...,990.0,3.0,2.0,0.0,...,2.0,3.0,25.0,13.0,0.0,2.2,1,1,0,0
41882,Parma,1,2022-08-12,Serie B,2022-2023,https://fbref.com/en/matches/be681acc/Parma-Ba...,990.0,2.0,1.0,0.0,...,1.0,3.0,24.0,22.0,0.0,2.1,1,1,0,1


In [4]:
##export_df = df.drop_duplicates(subset=['team', 'division', 'season'])

#export_df.to_csv("test.csv", index=False)

In [6]:
def apply_weighted_avg(col, match_date, match_red, current_match_date, decay_rate=0.005, time_window=365, min_games=5, recent_game_window=120):
    # Create a mask for non-NaN values
    valid_mask = ~pd.isna(col)
    
    # If all values are NaN, return NaN
    if not valid_mask.any():
        return np.nan
    
    # Filter out NaN values
    valid_col = col[valid_mask].copy()
    valid_dates = match_date[valid_mask]
    valid_red = match_red[valid_mask]
    
    # Create a time window mask (only include matches within time_window days BEFORE current match)
    time_window_mask = (current_match_date - valid_dates).dt.days <= time_window
    
    # If no matches in the time window, return NaN
    if not time_window_mask.any():
        return np.nan
    
    # Apply time window filter
    valid_col = valid_col[time_window_mask]
    valid_dates = valid_dates[time_window_mask]
    valid_red = valid_red[time_window_mask]
    
    # Check if we have minimum required games
    if len(valid_col) < min_games:
        return np.nan
    
    recent_games_mask = (current_match_date - valid_dates).dt.days <= recent_game_window
    if not recent_games_mask.any():
        return np.nan
    
    # Calculate weights for matches within the time window (relative to current match date)
    match_weight = np.exp(-(current_match_date - valid_dates).dt.days * decay_rate)
    
    # Reduce weight for matches with red cards
    match_weight = np.where(valid_red == 1, match_weight * 0.3, match_weight)
    
    # Ensure valid_col is numeric
    try:
        valid_col = pd.to_numeric(valid_col)
    except:
        return np.nan
    
    # Calculate weighted average
    weighted_avg = np.sum(match_weight * valid_col.values) / np.sum(match_weight)
    return weighted_avg

# Clean and prepare data
df = df.drop_duplicates(subset=['match_url', 'team'])
df['match_date'] = pd.to_datetime(df['match_date'])
df['match_red'] = df["summary_cards_red"].astype(int) + df["opp_summary_cards_red"].astype(int)
df['match_id'] = df['match_url']

# Automatically identify all numeric columns to process
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove columns we don't want to process
exclude_cols = ['match_red', 'summary_cards_red', 'opp_summary_cards_red', 'is_home']
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

# Separate team stats (without opp_ prefix) and opponent stats (with opp_ prefix)
team_stats = [col for col in numeric_cols if not col.startswith('opp_')]
opp_stats = [col for col in numeric_cols if col.startswith('opp_')]

print(f"Processing {len(team_stats)} team stats and {len(opp_stats)} opponent stats")

# Process each team
team_metrics = []

for team_name in df['team'].unique():
    team_matches = df[df['team'] == team_name].sort_values('match_date')
    
    for i, (idx, current_match) in enumerate(team_matches.iterrows()):
        if i > 0:  # Skip first match (no previous data)
            prev_matches = team_matches.iloc[:i]
            
            # NEW: Filter out matches from different divisions than current match
            # Only include previous matches that are in the same division as current match
            same_division_mask = prev_matches['division'] == current_match['division']
            prev_matches_same_div = prev_matches[same_division_mask]
            
            # If no matches in same division, skip this match (rolling averages will be null)
            if len(prev_matches_same_div) == 0:
                metrics_dict = {}
                
                # Set all rolling averages to NaN
                for stat in team_stats:
                    metrics_dict[f'team_rolling_{stat}'] = np.nan
                
                for stat in opp_stats:
                    clean_stat = stat.replace('opp_', '')
                    metrics_dict[f'team_rolling_conceded_{clean_stat}'] = np.nan
                
                # Add match info
                metrics_dict['team'] = team_name
                metrics_dict['match_id'] = current_match['match_id']
                metrics_dict['opp_team'] = current_match['opp_team']
                
                team_metrics.append(metrics_dict)
                continue
            
            metrics_dict = {}
            
            # Calculate rolling averages for all team stats (using only same division matches)
            for stat in team_stats:
                weighted_avg = apply_weighted_avg(
                    prev_matches_same_div[stat],
                    prev_matches_same_div['match_date'],
                    prev_matches_same_div['match_red'],
                    current_match['match_date']  # Pass current match date as reference
                )
                metrics_dict[f'team_rolling_{stat}'] = weighted_avg
            
            # Calculate rolling averages for all opponent stats (defensive perspective)
            for stat in opp_stats:
                weighted_avg = apply_weighted_avg(
                    prev_matches_same_div[stat],
                    prev_matches_same_div['match_date'],
                    prev_matches_same_div['match_red'],
                    current_match['match_date']  # Pass current match date as reference
                )
                # Remove 'opp_' prefix and add 'team_rolling_conceded_' prefix
                clean_stat = stat.replace('opp_', '')
                metrics_dict[f'team_rolling_conceded_{clean_stat}'] = weighted_avg
            
            # Add match info
            metrics_dict['team'] = team_name
            metrics_dict['match_id'] = current_match['match_id']
            metrics_dict['opp_team'] = current_match['opp_team']
            
            team_metrics.append(metrics_dict)

# Convert to dataframe
metrics_df = pd.DataFrame(team_metrics)

# Merge with original data
final_df = df.merge(
    metrics_df,
    on=['team', 'match_id', 'opp_team'],
    how='left'
)

# Create opponent metrics by swapping team/opp_team
opp_metrics_df = metrics_df.copy()
opp_metrics_df = opp_metrics_df.rename(columns={
    'team': 'opp_team',
    'opp_team': 'team'
})

# Rename all rolling columns to have opp_ prefix
rolling_cols = [col for col in opp_metrics_df.columns if col.startswith('team_rolling_')]
rename_dict = {col: f'opp_{col}' for col in rolling_cols}
opp_metrics_df = opp_metrics_df.rename(columns=rename_dict)

# Merge opponent metrics
final_df = final_df.merge(
    opp_metrics_df,
    on=['team', 'match_id', 'opp_team'],
    how='left'
)

# Select key columns for the final features table
key_cols = ['match_url', 'match_date', 'season', 'division', 'team', 'opp_team', 'is_home', 'gw', 'is_promoted?', 'opp_is_promoted?', 'is_early_season?']
rolling_cols = [col for col in final_df.columns if 'rolling' in col]

feature_cols = key_cols + rolling_cols
features_df = final_df[feature_cols].copy()

features_df

Processing 98 team stats and 96 opponent stats


KeyboardInterrupt: 

In [ ]:
features_df.to_sql('team_all_features_365_005_no_div_changes', conn, if_exists='replace', index=False)

conn.close()